In [ ]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['SPELLUSDT', 'AXSUSDT', 'ETHUSDT', 'BNBUSDT', 'OGNUSDT', 'CYBERUSDT', '1000PEPEUSDT', 'COMPUSDT', 'ADAUSDT', 'BCHUSDT', 'RUNEUSDT', 'MATICUSDT', 'BLZUSDT', 'HBARUSDT', 'TRBUSDT', 'WLDUSDT', 'MTLUSDT', 'BTCUSDT', 'XRPUSDT', 'OPUSDT', 'PERPUSDT', 'ARBUSDT', 'STMXUSDT', 'CFXUSDT', 'SOLUSDT', 'DOGEUSDT', 'APEUSDT', 'UNFIUSDT', 'LTCUSDT', 'STORJUSDT', 'CRVUSDT']
#lista=['STMXUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        #data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        data = md.estrategia_haz(symbol, debug=debug)
        #data,porcentajeentrada = md.estrategia_oro('XAU',tp_flag = True)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = imprimo)
        #resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        ########################################################################
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if data.disparo.iloc[-1]!=0: # imprime las monedas que estan en posible haz martillo
        #        print(symbol)
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


In [6]:
import modulos as md
import numpy as np
from backtesting import Backtest
import pandas_ta as ta
from backtesting import Strategy
import pandas as pd

def backtesting(data, plot_flag=False):
    balance = 100    
    def pivot(df_campo):
        indi=pd.Series(df_campo)
        return indi.to_numpy()
    class Fenix(Strategy):
        def init(self):
            super().init()
            self.pivot_high = self.I(pivot,self.data.pivot_high)
            self.pivot_low = self.I(pivot,self.data.pivot_low)
        def next(self):       
            super().next()
            if self.position:
                if self.data.cierra[-1]==True:
                    self.position.close()                    
            else:   
                if np.isnan(data.take_profit[-1]):
                    tp_value = None
                else:
                    tp_value = self.data.take_profit[-1]
                size= balance*self.data.porcentajeentrada[-1]/100
                if self.data.signal[-1]==1:
                    self.buy(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
                elif self.data.signal[-1]==-1:
                    self.sell(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
    bt = Backtest(data, Fenix, cash=balance)
    output = bt.run()
    if plot_flag:
        bt.plot()
    return output

####################################################################################

def detect_pivot_points(df):
    # Función para detectar los puntos pivote High y Low
    # Parámetros de longitud para los puntos pivote High y Low
    leftLenH = 5
    rightLenH = 5
    leftLenL = 5
    rightLenL = 5
    df['pivot_high'] = np.NaN
    df['pivot_low'] = np.NaN
    df['row_number'] = (range(len(df)))
    df.set_index('row_number', inplace=True)
    ## PIVOTS SUPERIORES
    for i in range(leftLenH, len(df) - rightLenH):
        if (
            df['High'].iloc[i] == df['High'].iloc[i - leftLenH:i + rightLenH + 1].max()
            and df['High'].iloc[i] > df['High'].iloc[i - 1]
            and df['High'].iloc[i] > df['High'].iloc[i + 1]
            ):
            df.at[i, 'pivot_high'] = df['High'].iloc[i]
    ## PIVOTS INFERIORES
    for i in range(leftLenL, len(df) - rightLenL):
        if (
            df['Low'].iloc[i] == df['Low'].iloc[i - leftLenL:i + rightLenL + 1].min()
            and df['Low'].iloc[i] < df['Low'].iloc[i - 1]
            and df['Low'].iloc[i] < df['Low'].iloc[i + 1]
            ):
            df.at[i, 'pivot_low'] = df['Low'].iloc[i]
    ## relleno
    for i in range(0, len(df) - 1):
        if np.isnan(df['pivot_low'].iloc[i]):
            df.at[i, 'pivot_low'] = df['pivot_low'].iloc[i - 1]
        if np.isnan(df['pivot_high'].iloc[i]):
            df.at[i, 'pivot_high'] = df['pivot_high'].iloc[i - 1]
    ## BOS
    df['bos'] = np.where(df.Close>df.pivot_high.shift(1),1,0)
    ## indice
    df['timestamp']=df['Open Time']
    df.set_index('timestamp', inplace=True)

##########################################################################################

def estrategia_real(symbol,debug = False):
    try:                
        np.seterr(divide='ignore', invalid='ignore')
        # temporalidad de 1h para encontrar martillos y soportes/resistencias
        data = md.obtiene_historial(symbol,'15m')      
        detect_pivot_points(data)          
        data.drop(['ema20','ema50', 'ema200'], axis=1, inplace=True)    
        data['signal'] = 0
        data['take_profit'] = np.NaN
        data['stop_loss'] = np.NaN
        data['cierra'] = False
        # Reemplazar valores no finitos (NA e inf) con 0
        data['porcentajeentrada'] = np.nan_to_num((data.Close/data.atr), nan=0, posinf=0, neginf=0)
        # Aplicar np.floor y convertir a enteros
        data['porcentajeentrada'] = np.floor(data['porcentajeentrada']).astype(int)
        ####################### alertas y valores
        if debug:
            df_str = data[list(data.columns)].to_string(index=False)
            print(df_str)
        return data
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+" - symbol: "+symbol+"\n")
        pass

#######################################################################################################################

symbol='XRPUSDT'
data = estrategia_real(symbol)
backtesting(data, plot_flag=True)

for index, values in data.iterrows():
    if values.bos != 0:
        print(f"{values['Open Time']} - {values.Close} - {values.pivot_low} - {values.bos}")

data.tail(60)     

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


2023-09-07 16:00:00 - 0.5002 - 0.4973 - 1
2023-09-07 16:15:00 - 0.5004 - 0.4973 - 1
2023-09-07 16:30:00 - 0.5009 - 0.4973 - 1
2023-09-07 16:45:00 - 0.5005 - 0.4973 - 1
2023-09-07 20:45:00 - 0.5019 - 0.4973 - 1
2023-09-07 21:00:00 - 0.5038 - 0.4973 - 1
2023-09-07 21:15:00 - 0.5034 - 0.4973 - 1
2023-09-08 15:00:00 - 0.5014 - 0.4992 - 1
2023-09-08 17:30:00 - 0.5029 - 0.4998 - 1
2023-09-08 17:45:00 - 0.5037 - 0.4998 - 1
2023-09-08 18:00:00 - 0.5029 - 0.4998 - 1
2023-09-08 18:15:00 - 0.503 - 0.4998 - 1
2023-09-08 18:30:00 - 0.5033 - 0.4998 - 1
2023-09-08 18:45:00 - 0.5035 - 0.4998 - 1
2023-09-08 19:00:00 - 0.5034 - 0.4998 - 1
2023-09-08 19:15:00 - 0.5049 - 0.4998 - 1
2023-09-08 19:30:00 - 0.505 - 0.4998 - 1
2023-09-08 19:45:00 - 0.5047 - 0.4998 - 1
2023-09-08 20:00:00 - 0.5054 - 0.4998 - 1
2023-09-08 20:15:00 - 0.5057 - 0.4998 - 1
2023-09-08 20:30:00 - 0.5045 - 0.4998 - 1
2023-09-10 20:00:00 - 0.4983 - 0.4938 - 1
2023-09-10 20:15:00 - 0.4995 - 0.4938 - 1
2023-09-11 23:30:00 - 0.4744 - 0.463

,Open Time,Open,High,Low,Close,Volume,atr,pivot_high,pivot_low,bos,signal,take_profit,stop_loss,cierra,porcentajeentrada
2023-09-16 08:45:00,2023-09-16 08:45:00,0.5004,0.5011,0.4993,0.4999,6709158.0,0.001382,0.5022,0.4980,0,0,NaN,NaN,False,361
2023-09-16 09:00:00,2023-09-16 09:00:00,0.5000,0.5006,0.4998,0.4999,4302052.4,0.001340,0.5022,0.4980,0,0,NaN,NaN,False,372
2023-09-16 09:15:00,2023-09-16 09:15:00,0.4999,0.5008,0.4999,0.5003,4078671.9,0.001309,0.5022,0.4980,0,0,NaN,NaN,False,382
2023-09-16 09:30:00,2023-09-16 09:30:00,0.5004,0.5016,0.5002,0.5013,6953651.4,0.001315,0.5022,0.4980,0,0,NaN,NaN,False,381
2023-09-16 09:45:00,2023-09-16 09:45:00,0.5013,0.5018,0.4999,0.5001,6691799.1,0.001357,0.5022,0.4980,0,0,NaN,NaN,False,368
2023-09-16 10:00:00,2023-09-16 10:00:00,0.5001,0.5006,0.4994,0.5005,9338777.7,0.001346,0.5022,0.4980,0,0,NaN,NaN,False,371
2023-09-16 10:15:00,2023-09-16 10:15:00,0.5006,0.5010,0.5002,0.5008,2524585.3,0.001307,0.5022,0.4980,0,0,NaN,NaN,False,383
2023-09-16 10:30:00,2023-09-16 10:30:00,0.5008,0.5017,0.5007,0.5011,3022799.9,0.001285,0.5022,0.4980,0,0,NaN,NaN,False,389
2023-09-16 10:45:00,2023-09-16 10:45:00,0.5011,0.5021,0.5010,0.5017,6154261.1,0.001272,0.5021,0.4980,0,0,NaN,NaN,False,394
2023-09-16 11:00:00,2023-09-16 11:00:00,0.5018,0.5020,0.5008,0.5015,6167505.9,0.001267,0.5021,0.4980,0,0,NaN,NaN,False,395
